In [1]:
import random, os, time
from collections import OrderedDict
from typing import Dict, Tuple, List, Union

import numpy as np

from psipy.rl.plant import Action, State, Plant
from psipy.rl.control.controller import Controller
from psipy.rl.control.controller import DiscreteRandomActionController
from psipy.rl.io.sart import SARTLogger
from psipy.rl.cycle_manager import CM


# Usage:
This notebook is a combination of "cycle_manager_toturial.ipython" and "episode_io_toturial.ipython". In order to make it happen, we will put time.sleep() in our loop.

In [2]:
class TestState(State):
    _channels = ("sensor1", "sensor2")

class TestAction(Action):
    dtype = "discrete"
    num_values = 1
    channels = ("act1", "act1/2")  # / will transform to | (pipe) since it conflicts with hdf5 group paths!
    legal_values = (range(101), range(101))

class TestPlant(Plant):
    state_type = TestState
    action_type = TestAction

    def check_initial_state(self, state: State) -> State:
        return self._get_next_state(TestState({"sensor1": 10, "sensor2":20}), None)

    def _get_next_state(self, state: State, action: Action) -> State:
        state.terminal = False
        state.reward = 1
        return TestState({"sensor1": 10, "sensor2": 20})

    def notify_episode_stops(self) -> bool:
        pass

plant = TestPlant()
name = "TutorialRun"
logdir = os.path.join(".", "test-logs")
max_steps = 100  # just how many loops we will do in the "episode"

In [6]:
plant.notify_episode_starts()
state = plant.check_initial_state(None)

drc = DiscreteRandomActionController(TestState.channels(), TestAction)

CM.notify_episode_starts(
    1,
    plant.__class__.__name__,
    "control",
)
sart = SARTLogger(logdir, name)
sart.notify_episode_starts()
new_file = True
for _ in range(33 * 10):  # 3s
    CM["loop"].tick()
    with CM["get-action"]:
        action = drc.get_action(state)
    with CM["get-state"]:
        next_state = plant.get_next_state(state, action)
        reward = plant.get_cost(next_state)
        terminal = plant.is_terminal(next_state)
    action_dict = action.as_dict(with_additional=True)
    data = dict(state=state.as_dict(), action=action_dict)
    CM.step(data)  # Increments step counter.

    with CM["sart-append"]:
        sart.append(data)
    state = next_state

    if terminal:
        data = dict(
            state=next_state.as_dict(),
            action=OrderedDict({k: np.nan for k in action.channels}),
        )
        sart.append(data)
        CM.step(data, increment_step_counter=False)
        break
    # User initiated episode stop / loop exit or max_steps reached.
    if CM.should_stop(max_steps=max_steps):
        break_all = CM.notify_episode_stops()
        break
        
    time.sleep(1/30)  # 30hz
    print("tock")
    CM["loop"].tock()

plant.notify_episode_stops()
sart.notify_episode_stops()

RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
PATH:
'state/values/sensor1'
PATH:
'state/values/sensor2'
PATH:
'state/cost'
PATH:
'state/terminal'
PATH:
'action/act1'
PATH:
'action/act1|2'
PATH:
'action/act1_index'
PATH:
'action/act1|2_index'
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock


Exception ignored in: <function ExpandableDataset.__del__ at 0x16837e160>
Traceback (most recent call last):
  File "/Users/slange/Code/sabbatical/dm-psiori/psipy/psipy/rl/io/sart.py", line 118, in __del__
    self.finalize()
  File "/Users/slange/Code/sabbatical/dm-psiori/psipy/psipy/rl/io/sart.py", line 153, in finalize
    self._resize(self.rows)
  File "/Users/slange/Code/sabbatical/dm-psiori/psipy/psipy/rl/io/sart.py", line 127, in _resize
    raise e
  File "/Users/slange/Code/sabbatical/dm-psiori/psipy/psipy/rl/io/sart.py", line 124, in _resize
    self.dataset.resize((rows, *self.incoming_shape))
  File "/Users/slange/Code/sabbatical/dm-psiori/.venv/lib/python3.8/site-packages/h5py/_hl/dataset.py", line 659, in resize
    self.id.set_extent(size)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5d.pyx", line 277, in h5py.h5d.DatasetID.set_extent
ValueError: Invalid datas

tock
tock
tock
tock
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
tock
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)
RESIZE!
(100,)


In [7]:
import h5py

files = os.listdir("test-logs/")
for filename in files:
    try:
        f = h5py.File(os.path.join("test-logs/", filename), 'r')
        print(f"file {filename}")
        print(f.keys())
        print(f['state'].keys())
        print(f['state']['values'].keys())
        print(f['state']['values']['sensor2'][:].shape)
    except:
        print(f"file {filename} is corrupted")
        continue

file TutorialRun-240903-123836-0-00.h5
<KeysViewHDF5 ['action', 'state']>
<KeysViewHDF5 ['cost', 'terminal', 'values']>
<KeysViewHDF5 ['sensor1', 'sensor2']>
(100,)
file TutorialRun-240903-124347-0-00.h5
<KeysViewHDF5 ['action', 'state']>
<KeysViewHDF5 ['cost', 'terminal', 'values']>
<KeysViewHDF5 ['sensor1', 'sensor2']>
(100,)
file TutorialRun-240903-123637-0-00.h5
<KeysViewHDF5 ['action', 'state']>
<KeysViewHDF5 ['cost', 'terminal', 'values']>
<KeysViewHDF5 ['sensor1', 'sensor2']>
(1,)
file TutorialRun-240903-123805-0-00.h5
<KeysViewHDF5 ['action', 'state']>
<KeysViewHDF5 ['cost', 'terminal', 'values']>
<KeysViewHDF5 ['sensor1', 'sensor2']>
(100,)
file TutorialRun-240903-123906-0-00.h5
<KeysViewHDF5 ['action', 'state']>
<KeysViewHDF5 ['cost', 'terminal', 'values']>
<KeysViewHDF5 ['sensor1', 'sensor2']>
(100,)


In [10]:

f['state']['values'].keys()



<KeysViewHDF5 ['sensor1', 'sensor2']>